### K_B_DataFlame読み込み
### レーサー期別データ読み込み

In [1]:
import polars as pl
import os

folder_path = 'DataFrame_csv'
file_name = 'K_B_Data.csv'
file_path = os.path.join(folder_path, file_name)

df = pl.read_csv(file_path, encoding='shift-jis')

folder_path = 'racer_csv'
file_name = 'racer_data.csv'
file_path = os.path.join(folder_path, file_name)

racer_data = pl.read_csv(file_path, encoding='shift-jis', dtypes={"選手登番": pl.Int64})

# 期間の範囲を定義
period_ranges = {
    '20前期': (20200501, 20201031),
    '20後期': (20201101, 20210430),
    '21前期': (20210501, 20211031),
    '21後期': (20211101, 20220430),
    '22前期': (20220501, 20221031),
    '22後期': (20221101, 20230430),
    '23前期': (20230501, 20231031),
    '23後期': (20231101, 20240430),
    '24前期': (20240501, 20241031)
}

# 期間判定用の式を作成
period_expressions = []
for period, (start, end) in period_ranges.items():
    period_expressions.append(
        pl.when(
            (pl.col('算出期間（自）').cast(pl.Int64) >= start) & 
            (pl.col('算出期間（至）').cast(pl.Int64) <= end)
        )
        .then(pl.lit(period))
        .otherwise(pl.lit(None))
        .alias(f'temp_period_{period}')
    )

# 一時的な期間カラムを追加
racer_data = racer_data.with_columns(period_expressions)

# coalesceで最終的な算出期間カラムを作成
temp_period_cols = [f'temp_period_{period}' for period in period_ranges.keys()]
racer_data = racer_data.with_columns([
    pl.coalesce(temp_period_cols).alias('算出期間')
])

# 一時的なカラムを削除
racer_data = racer_data.drop(temp_period_cols)

# マージするカラムを定義
merge_columns = [
    '1コース複勝率','1コース平均スタートタイミング','1コース平均スタート順位',
    '2コース複勝率','2コース平均スタートタイミング','2コース平均スタート順位',
    '3コース複勝率','3コース平均スタートタイミング','3コース平均スタート順位',
    '4コース複勝率','4コース平均スタートタイミング','4コース平均スタート順位',
    '5コース複勝率','5コース平均スタートタイミング','5コース平均スタート順位',
    '6コース複勝率','6コース平均スタートタイミング','6コース平均スタート順位',
    '前期能力指数', '今期能力指数'
]

# 期間ごとのマージ処理
for period in period_ranges.keys():
    # 期間ごとのデータをフィルタリング
    period_data = racer_data.filter(pl.col('算出期間') == period)
    
    # カラム名の変更と必要なカラムの選択
    renamed_columns = {col: f"{period}_{col}" for col in merge_columns}
    period_data = period_data.select([
        '選手登番',
        *[pl.col(col).alias(renamed_columns[col]) for col in merge_columns]
    ])
    
    # マージ
    df = df.join(
        period_data,
        on='選手登番',
        how='left'
    )


/tmp/ipykernel_2603/4131949391.py:14: DeprecationWarning: The argument `dtypes` for `read_csv` is deprecated. It has been renamed to `schema_overrides`.
  racer_data = pl.read_csv(file_path, encoding='shift-jis', dtypes={"選手登番": pl.Int64})


### レーサー期別データから特徴量作成

In [2]:
def create_feature_columns(df: pl.DataFrame) -> pl.DataFrame:
    """
    艇番に基づいて新しい特徴量を作成する関数
    
    Args:
        df (pl.DataFrame): 入力データフレーム
        
    Returns:
        pl.DataFrame: 新しい特徴量が追加されたデータフレーム
    """
    # 年度期間のリスト
    periods = [
        "20前期", "20後期", "21前期", "21後期",
        "22前期", "22後期", "23前期", "23後期","24前期"
    ]
    
    # 新しい特徴量の種類
    feature_types = [
        ("コース別複勝率", "複勝率"),
        ("コース別スタートタイミング", "平均スタートタイミング"),
        ("コース平均スタート順位", "平均スタート順位")
    ]
    
    # 各行に対して新しい特徴量を計算
    for period in periods:
        for new_feature, orig_feature in feature_types:
            new_col_name = f"{period}{new_feature}"
            
            # 艇番に基づいて元の列から値を取得
            df = df.with_columns([
                pl.concat_str([
                    pl.lit(f"{period}_"),
                    pl.col("艇番"),
                    pl.lit(f"コース{orig_feature}")
                ]).map_elements(lambda x: df.get_column(x)[0]).alias(new_col_name)
            ])
    
    return df


# 新しい特徴量の作成
df = create_feature_columns(df)

# 結果の確認
df.head()

/tmp/ipykernel_2603/3516095519.py:30: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df = df.with_columns([
/tmp/ipykernel_2603/3516095519.py:30: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df = df.with_columns([
/tmp/ipykernel_2603/3516095519.py:30: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df = df.with_columns([
/tmp/ipykernel_2603/3516095519.py:30: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df = df.with_columns([
/tmp/ipykernel_2603/3516095519.py:30: MapWithoutReturnDtypeWarning: 

実着順,選手登番,展示,ｽﾀｰﾄﾀｲﾐﾝｸ,レースID,艇番,選手名,年齢,支部,体重,級別,全国勝率,全国2連率,当地勝率,当地2連率,モーターNO,モーター2連率,ボートNO,ボート2連率,開催地,レース番号,日付,20前期_1コース複勝率,20前期_1コース平均スタートタイミング,20前期_1コース平均スタート順位,20前期_2コース複勝率,20前期_2コース平均スタートタイミング,20前期_2コース平均スタート順位,20前期_3コース複勝率,20前期_3コース平均スタートタイミング,20前期_3コース平均スタート順位,20前期_4コース複勝率,20前期_4コース平均スタートタイミング,20前期_4コース平均スタート順位,20前期_5コース複勝率,20前期_5コース平均スタートタイミング,20前期_5コース平均スタート順位,…,24前期_4コース平均スタートタイミング,24前期_4コース平均スタート順位,24前期_5コース複勝率,24前期_5コース平均スタートタイミング,24前期_5コース平均スタート順位,24前期_6コース複勝率,24前期_6コース平均スタートタイミング,24前期_6コース平均スタート順位,24前期_前期能力指数,24前期_今期能力指数,20前期コース別複勝率,20前期コース別スタートタイミング,20前期コース平均スタート順位,20後期コース別複勝率,20後期コース別スタートタイミング,20後期コース平均スタート順位,21前期コース別複勝率,21前期コース別スタートタイミング,21前期コース平均スタート順位,21後期コース別複勝率,21後期コース別スタートタイミング,21後期コース平均スタート順位,22前期コース別複勝率,22前期コース別スタートタイミング,22前期コース平均スタート順位,22後期コース別複勝率,22後期コース別スタートタイミング,22後期コース平均スタート順位,23前期コース別複勝率,23前期コース別スタートタイミング,23前期コース平均スタート順位,23後期コース別複勝率,23後期コース別スタートタイミング,23後期コース平均スタート順位,24前期コース別複勝率,24前期コース別スタートタイミング,24前期コース平均スタート順位
i64,i64,f64,f64,i64,i64,str,i64,str,i64,str,f64,f64,f64,f64,i64,f64,i64,f64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,5011,7.03,0.12,2005012401,1,"""高木圭大""",22,"""長崎""",51,"""B1""",2.78,3.8,1.93,3.66,25,22.5,30,43.89,24,1,200501,40.0,0.2,3.8,0.0,0.16,4.0,14.3,0.14,2.8,7.1,0.16,3.4,4.3,0.17,3.5,…,0.13,2.5,0.0,0.14,2.9,0.0,0.17,2.6,45.0,42.0,40.0,0.2,3.8,44.4,0.18,2.9,50.0,0.14,3.3,16.6,0.18,3.6,50.0,0.19,3.5,41.6,0.13,2.2,76.9,0.14,3.2,70.0,0.16,2.4,28.5,0.1,2.1
2,4268,6.99,0.14,2005012401,2,"""和合俊介""",34,"""福岡""",58,"""B1""",4.38,23.16,5.82,41.18,45,40.0,57,36.96,24,1,200501,50.0,0.15,2.4,38.1,0.15,2.6,15.0,0.15,2.8,14.3,0.13,2.5,0.0,0.15,3.6,…,0.16,3.5,7.1,0.16,3.5,0.0,0.21,4.5,47.0,43.0,0.0,0.16,4.0,25.0,0.22,3.8,14.2,0.18,3.9,33.3,0.18,2.8,62.5,0.14,3.4,18.7,0.16,3.6,27.2,0.17,3.6,10.0,0.16,3.2,29.4,0.19,3.9
3,4562,6.95,0.23,2005012401,5,"""阪本勇介""",32,"""岡山""",54,"""B1""",5.11,26.89,5.33,29.17,35,34.15,73,34.74,24,1,200501,55.6,0.15,3.1,22.7,0.18,3.9,45.8,0.19,3.7,28.6,0.18,3.3,3.4,0.19,4.1,…,0.17,3.5,0.0,0.18,3.9,0.0,0.18,4.1,49.0,45.0,4.3,0.17,3.5,10.0,0.14,3.1,13.3,0.14,2.5,10.5,0.15,3.1,20.0,0.14,3.1,6.6,0.17,3.1,9.0,0.16,3.3,0.0,0.16,3.2,0.0,0.14,2.9
4,4143,7.03,0.17,2005012401,4,"""浦田晃嗣""",38,"""佐賀""",56,"""B1""",4.25,18.45,4.21,16.13,46,31.43,25,35.56,24,1,200501,55.6,0.12,2.1,29.4,0.15,2.8,23.5,0.18,3.5,15.8,0.15,2.6,6.7,0.15,3.0,…,0.12,2.2,10.0,0.15,3.0,9.5,0.14,3.3,46.0,45.0,7.1,0.16,3.4,7.1,0.14,3.2,11.1,0.18,3.0,10.5,0.18,3.5,12.5,0.13,2.5,14.2,0.18,3.9,29.4,0.11,2.7,25.0,0.11,2.0,27.2,0.13,2.5
5,4919,6.98,0.14,2005012401,3,"""吉田翔悟""",29,"""長崎""",52,"""B1""",4.15,17.39,3.5,11.76,44,30.0,63,34.59,24,1,200501,71.4,0.14,2.9,23.5,0.15,2.7,31.6,0.17,2.8,53.3,0.14,3.3,18.2,0.14,2.6,…,0.15,2.9,25.0,0.12,2.8,5.8,0.18,4.3,49.0,47.0,14.3,0.14,2.8,16.6,0.18,3.8,10.5,0.16,3.1,9.5,0.16,2.3,9.0,0.15,2.5,6.2,0.14,2.8,13.3,0.16,3.1,25.0,0.13,2.4,15.7,0.17,3.6


### データ前処理

In [3]:
# データフレーム df の '級別' 列の値を変換
df = df.with_columns(
    pl.when(pl.col("級別") == "A1").then(1).when(pl.col("級別") == "A2").then(2).when(pl.col("級別") == "B1").then(3).when(pl.col("級別") == "B2").then(4).otherwise(pl.col("級別")).alias("級別")
)

# 列の型変換
for col in ['展示','ｽﾀｰﾄﾀｲﾐﾝｸ', '全国勝率', '全国2連率', '当地勝率', '当地2連率', 'モーター2連率', 'ボート2連率']:
    df = df.with_columns(pl.col(col).cast(pl.Float64))

# 型変換
for col in ['体重', '年齢', '艇番', '選手登番', 'モーターNO', '開催地', '実着順', 'レースID']:
    df = df.with_columns(pl.col(col).cast(pl.Int64))


# 列削除 (方法1: リスト内包表記 + startswith)
cols_to_drop = ['レース番号', 'ボートNO','日付']
years = [str(year) for year in range(20, 25)]  # '20', '21', '22', '23', '24'
periods = ['前期', '後期']
courses = [str(course) for course in range(1, 7)]  # '1', '2', '3', '4', '5', '6'

# 動的に生成する列名のリスト
drop_cols_dynamic = []

# パターンにマッチする列名を生成
for year in years:
    for period in periods:
        # 24後期を除外
        if year == '24' and period == '後期':
            continue
        for course in courses:
            drop_cols_dynamic.append(f"{year}{period}_{course}コース複勝率")
            drop_cols_dynamic.append(f"{year}{period}_{course}コース平均スタートタイミング")
            drop_cols_dynamic.append(f"{year}{period}_{course}コース平均スタート順位")
        drop_cols_dynamic.append(f"{year}{period}_前期能力指数") #追加

# 最初に定義した cols_to_drop と結合
cols_to_drop.extend(drop_cols_dynamic)

# 削除実行 
try:
    df = df.drop(cols_to_drop)
except pl.exceptions.ColumnNotFoundError as e:
    print(f"エラー: {e}")
    # 存在しない列名を特定
    existing_columns = set(df.columns)
    missing_columns = [col for col in cols_to_drop if col not in existing_columns]
    print(f"存在しない列: {missing_columns}")

print(df.columns)

['実着順', '選手登番', '展示', 'ｽﾀｰﾄﾀｲﾐﾝｸ', 'レースID', '艇番', '選手名', '年齢', '支部', '体重', '級別', '全国勝率', '全国2連率', '当地勝率', '当地2連率', 'モーターNO', 'モーター2連率', 'ボート2連率', '開催地', '20前期_今期能力指数', '20後期_今期能力指数', '21前期_今期能力指数', '21後期_今期能力指数', '22前期_今期能力指数', '22後期_今期能力指数', '23前期_今期能力指数', '23後期_今期能力指数', '24前期_今期能力指数', '20前期コース別複勝率', '20前期コース別スタートタイミング', '20前期コース平均スタート順位', '20後期コース別複勝率', '20後期コース別スタートタイミング', '20後期コース平均スタート順位', '21前期コース別複勝率', '21前期コース別スタートタイミング', '21前期コース平均スタート順位', '21後期コース別複勝率', '21後期コース別スタートタイミング', '21後期コース平均スタート順位', '22前期コース別複勝率', '22前期コース別スタートタイミング', '22前期コース平均スタート順位', '22後期コース別複勝率', '22後期コース別スタートタイミング', '22後期コース平均スタート順位', '23前期コース別複勝率', '23前期コース別スタートタイミング', '23前期コース平均スタート順位', '23後期コース別複勝率', '23後期コース別スタートタイミング', '23後期コース平均スタート順位', '24前期コース別複勝率', '24前期コース別スタートタイミング', '24前期コース平均スタート順位']


### データフレーム保存

In [4]:
folder_path = 'DataFrame_csv'

# ファイルのパスをフォルダパスと結合
file_path = os.path.join(folder_path, 'DataFrame.csv')

# CSVデータを文字列として生成し、Shift-JISエンコーディングで保存
with open(file_path, "w", encoding="shift-jis") as fw:
    fw.write(df.write_csv())